In [1]:
!pip install espn_api


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [39]:
import pandas as pd
import numpy as np
import espn_api as espn
from espn_api.football import League
import matplotlib.pyplot as plt
import seaborn as sns

### Connect to league

In [58]:
import os
# from espn_api.football import League

LEAGUE_ID = 86952922
YEARS = [2021, 2022, 2024]

y = YEARS[-1]

# for y in YEARS[-1:]:
    
espn_s2 = "AEC20e998honXS4Wi0Z8qzlJdam4%2F%2BaApa7apspnhKR0Npb%2FMsF5DuQsFUcHW%2FhPihQun9U6PGITOi2CkbdfDCkUc8druBVhAwT08Lzrvv8oZli8YAuTi9mIWg7YqtorCNOEKPxHpYswnT3q7b885tRDKBJpLCH0T2h4h1p%2B02SfdlDhjEB2gHqFk1xl6tJRNMBiCkZ8i5RttLW6ER9ZvLTmmAdb5ceZhQ27NEMiMf%2BjWSSvwBdnf2roxwt9baw33BVnnITqYVb8FXsaUwm7%2Bm0m9GLQ%2B66%2BU%2Brg%2BQngXm1ekA%3D%3D"
swid = "{B431504E-F779-4C49-B3E8-28DDF7409957}"
kwargs = {"league_id": LEAGUE_ID, "year": y, "swid":swid, "espn_s2":espn_s2}
league = League(**kwargs)
teams_count = len(league.teams) if getattr(league, "teams", None) else None
# Try to read a small bit of data to validate
draft = league.draft

### Get historical fantasy stats from Pro Football Reference

In [83]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import re

def scrape_pfr_fantasy(year: int) -> pd.DataFrame:
    url = f"https://www.pro-football-reference.com/years/{year}/fantasy.htm"
    headers = {"User-Agent": "Mozilla/5.0"}
    html = requests.get(url, headers=headers).text
    soup = BeautifulSoup(html, "html.parser")

    # Helper: return the fantasy table element whether it's in DOM or inside comments
    def find_fantasy_table(soup: BeautifulSoup):
        # 1) try live
        t = soup.find("table", {"id": "fantasy"})
        if t:
            return t
        # 2) scan comments
        for c in soup.find_all(string=lambda t: isinstance(t, Comment)):
            if 'id="fantasy"' in c:
                frag = BeautifulSoup(c, "html.parser")
                t = frag.find("table", {"id": "fantasy"})
                if t:
                    return t
        return None

    table = find_fantasy_table(soup)
    if table is None:
        raise ValueError(f"Fantasy table not found for {year}")

    # Build column order from the LAST header row in thead (skip over_header rows)
    thead = table.find("thead")
    header_rows = [tr for tr in thead.find_all("tr") if "over_header" not in tr.get("class", [])]
    # last header row is the one with the actual columns
    last_hdr = header_rows[-1]
    cols = []
    for th in last_hdr.find_all("th"):
        key = th.get("data-stat", "").strip()
        if key:  # skip corner blanks
            cols.append(key)

    # Parse body rows using data-stat keys
    data = []
    tbody = table.find("tbody")
    for tr in tbody.find_all("tr"):
        # PFR sometimes repeats a header row inside tbody with class="thead"
        if "thead" in tr.get("class", []):
            continue

        row = {}
        # some tables have a row header in <th scope="row">
        th = tr.find("th", {"scope": "row"})
        if th is not None:
            k = th.get("data-stat", "").strip() or "rk"
            row[k] = th.get_text(strip=True)

        for td in tr.find_all("td"):
            k = td.get("data-stat", "").strip()
            if not k:
                continue
            row[k] = td.get_text(strip=True)

        # skip completely empty rows
        if not row:
            continue
        data.append(row)

    # Create DataFrame and align to column order (keep any extra keys too)
    df = pd.DataFrame(data)

    # Ensure the final column order starts with `cols` then any extras
    extras = [c for c in df.columns if c not in cols]
    df = df[[c for c in cols if c in df.columns] + extras]

    # Clean: remove repeated header rows if any sneaked in
    if "player" in df.columns:
        df = df[df["player"].str.lower() != "player"]

    # Coerce numeric columns
    for c in df.columns:
        if c in ("player", "team", "pos", "tm"):
            continue
        df[c] = pd.to_numeric(df[c], errors="ignore")

    return df

# Example usage
df_2021 = scrape_pfr_fantasy(2019)


In [84]:
pd.set_option('display.max_columns', None)
df_2021.head()

,ranker,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,pass_int,rush_att,rush_yds,rush_yds_per_att,rush_td,targets,rec,rec_yds,rec_yds_per_rec,rec_td,fumbles,fumbles_lost,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,draftkings_points,fanduel_points,vbd,fantasy_rank_pos,fantasy_rank_overall
0,1,Christian McCaffrey*+,CAR,RB,23,16,16,0,2,0,0,0,287,1387,4.83,15,142,116,1005,8.66,4,1.0,0,19,1.0,NaN,355.0,471.2,477.2,413.2,215.0,1,1.0
1,2,Lamar Jackson*+,BAL,QB,22,15,15,265,401,3127,36,6,176,1206,6.85,7,0,0,0,NaN,0,9.0,2,7,NaN,NaN,416.0,415.7,429.7,421.7,152.0,1,2.0
2,3,Derrick Henry*,TEN,RB,25,15,15,0,0,0,0,0,303,1540,5.08,16,24,18,206,11.44,2,5.0,3,18,NaN,NaN,277.0,294.6,303.6,285.6,136.0,2,3.0
3,4,Aaron Jones,GNB,RB,25,16,16,0,0,0,0,0,236,1084,4.59,16,68,49,474,9.67,3,3.0,2,19,NaN,NaN,266.0,314.8,322.8,290.3,125.0,3,4.0
4,5,Ezekiel Elliott*,DAL,RB,24,16,16,0,0,0,0,0,301,1357,4.51,12,71,54,420,7.78,2,3.0,2,14,NaN,NaN,258.0,311.7,319.7,284.7,117.0,4,5.0


In [85]:
df_2021['player_name'] = df_2021['player'].str.replace(r"[*+]", "", regex=True).str.strip()
stats = df_2021[['player_name', 'team', 'fantasy_pos', 'g', 'gs', 'fantasy_points_ppr']]
stats = stats.loc[stats['fantasy_pos'].isin(['QB', 'RB', 'WR', 'TE'])]
stats['fantasy_points_ppr'].fillna(0, inplace=True)
stats["pos_rank"] = (
    stats.groupby("fantasy_pos")["fantasy_points_ppr"]
      .rank(method="dense", ascending=False)
      .astype(int)
)

stats["full_rank"] = (
    stats["fantasy_points_ppr"]
      .rank(method="dense", ascending=False)
      .astype(int)
)

In [86]:
stats.sort_values('fantasy_points_ppr', ascending=False).head(20)

,player_name,team,fantasy_pos,g,gs,fantasy_points_ppr,pos_rank,full_rank
0,Christian McCaffrey,CAR,RB,16,16,471.2,1,1
1,Lamar Jackson,BAL,QB,15,15,415.7,1,2
6,Michael Thomas,NOR,WR,16,15,374.6,1,3
11,Dak Prescott,DAL,QB,16,16,337.8,2,4
13,Russell Wilson,SEA,QB,16,16,328.6,3,5
16,Deshaun Watson,HOU,QB,15,15,321.0,4,6
3,Aaron Jones,GNB,RB,16,16,314.8,2,7
4,Ezekiel Elliott,DAL,RB,16,16,311.7,3,8
10,Austin Ekeler,LAC,RB,16,8,309.0,4,9
29,Jameis Winston,TAM,QB,16,16,305.4,5,10


### Create Draft Value Metric 

In [87]:
def compute_vorp_star(
    df: pd.DataFrame,
    teams: int = 12,
    starters_per_team: dict = None,
    use_ppg: bool = False,
    min_games_for_ppg: int = 8,
    pool_factor: float = 2.0,         # pool ~ 2× starters per position
    winsor_limits: tuple = (0.02, 0.98)  # clamp extremes for robust scale
):
    """
    Returns a copy of df with columns:
      - points_used         (PPG or season total)
      - rep_points          (replacement level for the position)
      - vorp_raw            (points_used - rep_points)
      - pos_scale_robust    (1.4826 * MAD of (points_used - rep_points) in pool)
      - vorp_star           (vorp_raw / pos_scale_robust)
      - injury_flag         (True if < min_games_for_ppg when use_ppg)
    """

    df = df.copy()

    # 1) Decide points metric (total or PPG)
    if use_ppg:
        df["points_used"] = df["fantasy_points_ppr"] / df["g"].replace(0, np.nan)
        # flag small-sample seasons
        df["injury_flag"] = (df["g"] < min_games_for_ppg) | (~np.isfinite(df["points_used"]))
    else:
        df["points_used"] = df["fantasy_points_ppr"]
        df["injury_flag"] = False

    # Fill any NaNs created (e.g., 0 games) with 0 so later math works;
    # they’ll still be flagged as injury_flag=True if use_ppg
    df["points_used"] = df["points_used"].fillna(0.0)

    # 2) Replacement level per position
    # Default lineup for 12-team, 1QB/2RB/2WR/1TE (adjust if your league differs)
    if starters_per_team is None:
        starters_per_team = {"QB": 1, "RB": 2, "WR": 2, "TE": 1}

    # How many starters in the league per position = teams * starters_per_team[pos]
    rep_index = {
        pos: teams * starters_per_team.get(pos, 0)
        for pos in df["fantasy_pos"].unique()
    }

    # Helper: robust MAD scale
    def mad(x):
        med = np.median(x)
        return 1.4826 * np.median(np.abs(x - med))

    # Helper: winsorize
    def winsorize(s, lo=0.02, hi=0.98):
        ql, qh = np.quantile(s, [lo, hi])
        return np.clip(s, ql, qh)

    out_frames = []
    for pos, grp in df.groupby("fantasy_pos", sort=False):
        # sort high→low by points_used
        gsort = grp.sort_values("points_used", ascending=False)

        # replacement rank (e.g., RB24). If none defined, skip normalization gracefully.
        # R = rep_index.get(pos, 0)
        # if R <= 0 or R > len(gsort):
        #     # no valid replacement index; fall back to zero baseline/scale=1 to avoid div-by-0
        #     rep_points = 0.0
        # else:
        #     rep_points = gsort.iloc[R-1]["points_used"]
        
        if pos == "TE":
            # use average of TE13–TE20 as replacement
            tail = gsort.iloc[8:12]["points_used"]  # 0-based index → TE13 is row 12
            rep_points = tail.mean() if len(tail) else 0.0
        else:
            R = rep_index.get(pos, 0)
            if R <= 0 or R > len(gsort):
                rep_points = 0.0
            else:
                rep_points = gsort.iloc[R-1]["points_used"]


        gsort = gsort.assign(rep_points=rep_points)
        gsort = gsort.assign(vorp_raw=gsort["points_used"] - rep_points)

        # build pool ~ top (pool_factor × starters) to estimate robust spread
        pool_size = int(max(R * pool_factor, min(len(gsort), R))) if R > 0 else min(len(gsort), 24)
        pool = gsort.iloc[:pool_size]["vorp_raw"].values

        # winsorize pool then MAD
        pool_w = winsorize(pool, *winsor_limits) if len(pool) else np.array([0.0])
        scale = mad(pool_w)
        if scale == 0 or not np.isfinite(scale):
            # fallback: use IQR/1.349 (approximately SD under normality) or 1 to avoid divide-by-0
            q1, q3 = np.percentile(pool_w, [25, 75]) if len(pool_w) else (0.0, 0.0)
            iqr = q3 - q1
            scale = (iqr / 1.349) if iqr > 0 else 1.0

        gsort = gsort.assign(pos_scale_robust=scale)
        gsort = gsort.assign(vorp_star=gsort["vorp_raw"] / scale)

        out_frames.append(gsort)

    result = pd.concat(out_frames, axis=0).sort_index()

    # Optional: ranks
    result["vorp_star_rank_overall"] = result["vorp_star"].rank(method="dense", ascending=False).astype(int)
    result["vorp_star_rank_pos"] = (
        result.groupby("fantasy_pos")["vorp_star"].rank(method="dense", ascending=False).astype(int)
    )

    return result

# -------------------------
# Example usage on your df:
# -------------------------
# df has: player_name, team, fantasy_pos, g, gs, fantasy_points_ppr, pos_rank, full_rank

# Example for 12-team, 1QB/2RB/2WR/1TE, using season totals:
df_v = compute_vorp_star(
    stats,
    teams=12,
    starters_per_team={"QB": 1, "RB": 2, "WR": 2, "TE": 1},
    use_ppg=False,           # set True if you prefer per-game
    min_games_for_ppg=8,
    pool_factor=2.0,
    winsor_limits=(0.02, 0.98),
)

# Columns now available:
# ['points_used','rep_points','vorp_raw','pos_scale_robust','vorp_star',
#  'injury_flag','vorp_star_rank_overall','vorp_star_rank_pos', ...]
# Example view:
cols = [
    "player_name","team","fantasy_pos","fantasy_points_ppr","vorp_star","vorp_star_rank_overall", "vorp_star_rank_pos"
]
df_v[cols].sort_values(["vorp_star"], ascending=False).head(20)


,player_name,team,fantasy_pos,fantasy_points_ppr,vorp_star,vorp_star_rank_overall,vorp_star_rank_pos
0,Christian McCaffrey,CAR,RB,471.2,4.358471,1,1
7,Travis Kelce,KAN,TE,254.3,3.717194,2,1
1,Lamar Jackson,BAL,QB,415.7,3.654994,3,1
6,Michael Thomas,NOR,WR,374.6,3.336522,4,1
15,George Kittle,SFO,TE,222.5,2.657993,5,2
19,Darren Waller,OAK,TE,221.0,2.608031,6,3
21,Zach Ertz,PHI,TE,215.6,2.428167,7,4
12,Mark Andrews,BAL,TE,207.2,2.148378,8,5
3,Aaron Jones,GNB,RB,314.8,2.092298,9,2
4,Ezekiel Elliott,DAL,RB,311.7,2.047380,10,3
